In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import math
import numpy as np
import json
from copy import deepcopy
from config import filename
print(filename)

data/smallRandom.json


In [26]:
def n2letter(n):
    '''0 to 'a', 1 to 'b', ... '''
    return chr(96+n)

def string2duration(string):
    ''' "01:50:19.3177493" to duration in seconds'''
    return 3600*int(string[:2]) + 60*int(string[3:5]) + float(string[6:])

In [27]:
def read_data(path):
    global task_count
    global data
    global tasks
    file = open(path)
    data = json.load(file)
    tasks = data['nodes']
    for task in tasks:
        tasks[task]['Data'] = string2duration(tasks[task]['Data'])
    task_count = len(tasks)
    print("Data loaded successfully. Number of tasks: " + str(task_count))

n_cores = 2
read_data(filename)
tasks

Data loaded successfully. Number of tasks: 10


{'1': {'Data': 2843.6552354, 'Dependencies': []},
 '2': {'Data': 3656.0712509, 'Dependencies': [1]},
 '3': {'Data': 2741.9822376, 'Dependencies': [1]},
 '4': {'Data': 4166.224333, 'Dependencies': [1]},
 '5': {'Data': 5065.7910236, 'Dependencies': [2]},
 '6': {'Data': 5116.0078169, 'Dependencies': [4]},
 '7': {'Data': 3878.7031112, 'Dependencies': [2]},
 '8': {'Data': 3596.1902359, 'Dependencies': [5]},
 '9': {'Data': 5252.579534, 'Dependencies': [7]},
 '10': {'Data': 2883.6393571, 'Dependencies': [8]}}

In [28]:
#Tasks to child tasks / Tasks to parents / Task is terminal / Task is inital
task2childs = {task : list() for task in tasks}
task2parents = {task : list() for task in tasks}
for task, info in tasks.items():
    #Add childs
    list_task_parents = info['Dependencies']
    for task_parent in list_task_parents:
        task2childs[str(task_parent)].append(task)
    #Add parents
    task2parents[str(task)] = tasks[task]['Dependencies']
    
def task_is_terminal(task: int):
    return len(task2childs[str(task)]) == 0
def task_is_inital(task: int):
    return len(task2parents[str(task)]) == 0

# print(task2childs)
# print(task2parents)

In [29]:
task2sbl = {}

def save_static_bottom_level(task : int):
    task_duration = tasks[str(task)]["Data"]
    if task_is_terminal(task):
        sbl = task_duration
    else:
        list_sbl_child = list()
        for task_child in task2childs[str(task)]:
            if task_child in task2sbl:
                sbl_child = task2sbl[task_child]
            else:
                sbl_child = save_static_bottom_level(task_child)
            list_sbl_child.append(sbl_child)
        sbl = max(list_sbl_child) + task_duration
                
    task2sbl[task] = sbl
    return sbl

for task in tasks:
    if task_is_inital(task):
        save_static_bottom_level(task)
        
task2sbl



{'10': 2883.6393571,
 '8': 6479.829593,
 '5': 11545.620616600001,
 '9': 5252.579534,
 '7': 9131.2826452,
 '2': 15201.691867500002,
 '3': 2741.9822376,
 '6': 5116.0078169,
 '4': 9282.232149899999,
 '1': 18045.347102900003}

In [30]:
class Node():

    def __init__(self, parent = None, task_to_add = None, core_where_to_add = None, time_task_start = None, graph = None):
        '''Create a Node object ie a partial scheduling
        parent = parent Node, None if root
        task_to_add : task added to the partial schedule
        core_where_to_add : core where to do task
        time_task_start : instant where the core will start computing the task
        '''
        
        if parent is None:
            self.tasks = graph
            self.schedule = {core : list() for core in range(n_cores)}              #lists of the tasks made on each core, ordered by most recent tasks
            self.cores_available_time = {core : 0 for core in range(n_cores)}       #instant t where core will be available
            self.tasks_done_time = {}                                               #instant t where task was done for each task done
            self.tasks_started_time = {}                                            #instant t where task was started for each task done
                        
        else:
            self.tasks = parent.tasks
            
            self.schedule = deepcopy(parent.schedule)
            self.schedule[core_where_to_add].append(task_to_add)
            
            duration_task = self.tasks[task_to_add]['Data']
            self.cores_available_time = deepcopy(parent.cores_available_time)
            self.cores_available_time[core_where_to_add] = time_task_start + duration_task
            
            self.tasks_started_time = deepcopy(parent.tasks_started_time)
            self.tasks_started_time[task_to_add] = time_task_start 
            
            self.tasks_done_time = deepcopy(parent.tasks_done_time)
            self.tasks_done_time[task_to_add] = time_task_start + duration_task            
    
    def __repr__(self):
        string = ''
        for core_schedule in self.schedule.values():
            for task in core_schedule:
                string += n2letter(int(task))
            string += '|'
        return f"{string[:-1]} --- Cores available at time: {self.cores_available_time}. Tasks started times : {self.tasks_started_time}. Tasks done times : {self.tasks_done_time}"
            
    def is_terminal(self):
        '''Return whether a node is a full schedule'''
        return len(self.tasks_done_time) == task_count
    
    def succ(self):                     
        '''Create and return list of child node of self'''
        childs = list()
        
        #On regarde toutes les tâches qu'on va tenter de rajouter
        for task, info in self.tasks.items():
            if task in self.tasks_done_time: #On passe les taches déjà ajoutées
                continue
            if not all([str(task_required) in self.tasks_done_time for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
                continue
            
            time_all_dependencies_done = max([0] + [self.tasks_done_time[str(task_required)] for task_required in self.tasks[task]['Dependencies']])
            
            for core in range(n_cores):
                time_core_available = self.cores_available_time[core]
                time_task_start = max(time_core_available, time_all_dependencies_done)  #We wait that the core end its tasks AND that every dependencies was done.
                child = Node(parent = self, task_to_add=task, core_where_to_add=core, time_task_start=time_task_start)
                childs.append(child)
                
        return childs
        
    def cost(self, child_node):
        '''Return the cost of going from self to child_node, a child node of self
        '''
        g_self = max(self.cores_available_time.values())
        g_child = max(child_node.cores_available_time.values())
        cost = g_child - g_self
        return cost
    
    def compute_g(self):
        return max(self.cores_available_time.values())
    
root = Node(graph = tasks)
a = root.succ()[0]
a.succ()

[ab| --- Cores available at time: {0: 6499.7264863, 1: 0}. Tasks started times : {'1': 0, '2': 2843.6552354}. Tasks done times : {'1': 2843.6552354, '2': 6499.7264863},
 a|b --- Cores available at time: {0: 2843.6552354, 1: 6499.7264863}. Tasks started times : {'1': 0, '2': 2843.6552354}. Tasks done times : {'1': 2843.6552354, '2': 6499.7264863},
 ac| --- Cores available at time: {0: 5585.637473000001, 1: 0}. Tasks started times : {'1': 0, '3': 2843.6552354}. Tasks done times : {'1': 2843.6552354, '3': 5585.637473000001},
 a|c --- Cores available at time: {0: 2843.6552354, 1: 5585.637473000001}. Tasks started times : {'1': 0, '3': 2843.6552354}. Tasks done times : {'1': 2843.6552354, '3': 5585.637473000001},
 ad| --- Cores available at time: {0: 7009.8795684, 1: 0}. Tasks started times : {'1': 0, '4': 2843.6552354}. Tasks done times : {'1': 2843.6552354, '4': 7009.8795684},
 a|d --- Cores available at time: {0: 2843.6552354, 1: 7009.8795684}. Tasks started times : {'1': 0, '4': 2843.65

In [31]:
def h(node):
    '''Estimated remaining time of the node-schedule for reaching a terminal node.
    Must understimate true value.'''
    successor_tasks = list()
    for task, info in tasks.items():
        if task in node.tasks_done_time: #On passe les taches déjà ajoutées
            continue
        if not all([str(task_required) in node.tasks_done_time for task_required in info['Dependencies']]):   #On ne garde que les taches dont toutes les dépendances ont été réalisées
            continue
        successor_tasks.append(task)
    if successor_tasks == []:
        return 0
    return max([task2sbl[task] for task in successor_tasks])

h(root), h(a)

(18045.347102900003, 15201.691867500002)

In [32]:


class A_star():
    def __init__(self, root):
        self.root = root
    
    def find_best_path(self):
        root.g = 0
        root.f = root.g + h(root)
        self.closed_list = []
        self.open_list = [root]
        while len(self.open_list) > 0:
            next_node_to_explore = self.open_list.pop(0)
            self.closed_list.append(next_node_to_explore)
            if next_node_to_explore.is_terminal():
                print("Result:", next_node_to_explore)
                return 
            for node_child in next_node_to_explore.succ():
                # node_child.g = next_node_to_explore.g + next_node_to_explore.cost(node_child)
                node_child.g = node_child.compute_g() #Other method, doesnt work on all Node classes
                node_child.f = node_child.g + h(node_child)
                self.open_list.append(node_child)
            self.open_list.sort(key = lambda node : node.f)
            
        raise Exception("No path from root to a terminal node")

root = Node(graph=tasks)
a_star = A_star(root = root)
a_star.find_best_path()

Result: abehf|dgijc --- Cores available at time: {0: 20277.7155627, 1: 21766.783808300002}. Tasks started times : {'1': 0, '2': 2843.6552354, '4': 2843.6552354, '5': 6499.7264863, '7': 7009.8795684, '8': 11565.5175099, '9': 10888.5826796, '6': 15161.7077458, '10': 16141.1622136, '3': 19024.8015707}. Tasks done times : {'1': 2843.6552354, '2': 6499.7264863, '4': 7009.8795684, '5': 11565.5175099, '7': 10888.5826796, '8': 15161.7077458, '9': 16141.1622136, '6': 20277.7155627, '10': 19024.8015707, '3': 21766.783808300002}
